### 解释逻辑回归的结果

在这个 notebook 与 测试题中，你将做一些解释逻辑回归系数的练习。在前面的视频中看到的内容，应该有助于你完成该 notebook 。

该数据集包含四个变量： `admit`、 `gre`、 `gpa` 与 `prestige`

* `admit` 是一个二元变量，它表明一个候选人是否被 UCLA 录取，录取=1，不被录取=0 。
* `gre` 代表 GRE 成绩，GRE 指的是研究生入学考试。
* `gpa` 代表平均学分绩点 （Grade Point Average）。
* `prestige` 代表申请人母校（此次申请之前申请人曾就读的学校）的声誉，其中1代表最高（声誉最高），4代表最低（没有声誉）。

首先，让我们导入必要的库和数据。

In [27]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv("./admissions.csv")
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


这里有几种不同的方法，你可以选择其中一个在这个数据集中使用 `prestige` 列。对于这个数据集，我们希望从声誉1到声誉2，有一个录取率；从声誉3变为声誉4，有一个不同的录取率。

1. 有了上述的想法之后，创建几个虚拟变量，这几个变量可以把声誉改成一个分类变量，而不是定量，然后回答下面的测试1。

2. 现在，拟合一个逻辑回归模型，使用 `gre`、 `gpa` 以及使用基准值为 `1`的  `prestige` 来预测一个人是否会被录取。根据你的结果来回答下面的测试2和3。 不要忘记添加一个截距。

In [28]:
# 众数
df.mode()

,admit,gre,gpa,prestige
0,0,620,4.0,2


In [29]:
# 此处代码框方式不对，被这句：我们希望从声誉1到声誉2，有一个录取率；从声誉3变为声誉4，有一个不同的录取率。误导了
'''
# 转化 prestige 为 0，1
bin_edges = [0,2,4]
bin_names = [0,1]
df['prestigenew'] = pd.cut(df['prestige'], bin_edges, labels = bin_names)
df.head()

# 检查转换后的数量
df.groupby('prestigenew').size()

# 修改prestigenew的类型
df['prestigenew'] = df['prestigenew'].astype('int')
df.dtypes

# 适配
df['intercept'] =1
lm = sm.Logit(df['admit'],df[['intercept','gre','gpa','prestigenew']])
result = lm.fit()
result.summary()
'''

,admit,gre,gpa,prestige,prestigenew
0,0,380,3.61,3,1
1,1,660,3.67,3,1
2,1,800,4.00,1,0
3,1,640,3.19,4,1
4,0,520,2.93,4,1


In [52]:
# 题目中的意思是看如果别的条件不变化，prestige的变化对录取率的影响
# 建立4个虚拟变量
df[['pre_1','pre_2','pre_3','pre_4']] = pd.get_dummies(df['prestige'])
df.head()

,admit,gre,gpa,prestige,prestigenew,intercept,pre_1,pre_2,pre_3,pre_4
0,0,380,3.61,3,1,1,0,0,1,0
1,1,660,3.67,3,1,1,0,0,1,0
2,1,800,4.00,1,0,1,1,0,0,0
3,1,640,3.19,4,1,1,0,0,0,1
4,0,520,2.93,4,1,1,0,0,0,1


In [53]:
# 适配
# 因为题目中是要和pre1的变化比较，所以这里排除pre1
# 注意这里要用到逻辑分类Logit进行回归
df['intercept'] =1
lm = sm.Logit(df['admit'],df[['intercept','gre','gpa','pre_2','pre_3','pre_4']])
result = lm.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 21 Dec 2018   Pseudo R-squ.:                 0.08166
Time:                        11:19:04   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -3.8769      1.142     -3.393      0.001      -6.116      -1.638
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
pre_2         -0.6801      0.317     -2.146      0.032      -1.301      -0.059
pre_3         -1.3387      0.345     -3.882      0.000      -2.015      -0.663
pre_4         -1.5534      0.417     -3.721      0.000      -2.372      -0.735
==============================================================================
"""

In [54]:
# 计算 gpa 倍率
# 当 gpa 上升一个单位的时候，倍率的变化
np.exp(0.7793)

2.1799457692483717

In [42]:
# 计算 pre2 倍率
# 因为从 pre1 上升到 pre2 录取率是下降了
# 根据问题，这里要回答的是pre2变为pre1，所以取倒数：
1/np.exp(-0.6801)

1.9740751298733885

In [43]:
# 计算 pre3 倍率
# 同上
1/np.exp(-1.3387)

3.81408197450317

In [44]:
# 计算 pre4 倍率
# 同上
1/np.exp(-1.5534)

4.727516444398727

In [55]:
# 对比，如果滤掉了pre4的话，结果和意义如下
df['intercept'] =1
lm = sm.Logit(df['admit'],df[['intercept','gre','gpa','pre_1','pre_2','pre_3']])
result = lm.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 21 Dec 2018   Pseudo R-squ.:                 0.08166
Time:                        11:20:52   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     -5.4303      1.140     -4.764      0.000      -7.664      -3.196
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
pre_1          1.5534      0.417      3.721      0.000       0.735       2.372
pre_2          0.8733      0.367      2.378      0.017       0.153       1.593
pre_3          0.2147      0.393      0.547      0.584      -0.555       0.984
==============================================================================
"""

In [49]:
# 如果从pre1，变为pre2，录取率为0.212
# 这个倒数正好对应前面结果的倍数
1/np.exp(1.5534),np.exp(1.5534)

(0.21152755611983615, 4.727516444398727)

In [50]:
# 同上
1/np.exp(0.8733),np.exp(0.8733)

(0.41757128781906594, 2.394800670378709)

In [51]:
# 同上
1/np.exp(0.2147),np.exp(0.2147)

(0.8067834389073746, 1.2394899941851785)